In [ ]:
import glob
import math

import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches

In [ ]:
# Specify ISS USOS module geometry "keepin boxes"
# From https://babelfish.arc.nasa.gov/bitbucket/projects/ASTROBEE/repos/astrobee_ops/browse/gds/ControlStationConfig/IssWorld-ISAAC/keepins/keepin.json
KEEPINS = [ [ -2.3920228000000012, -0.638923, 4.190492, -0.9758328000000001, 0.681877, 5.426472 ],
                  [ -0.9758328000000001, -1.044933, 3.7924819999999997, 5.892357199999999, 1.085177, 5.915652 ],
                  [ 5.892357199999999, -0.653923, 4.190492, 6.8085472, 0.6668769999999999, 5.426472 ],
                  [ 10.272166400000001, 1.8509731999999999, 4.189467199999999, 11.5929664, 2.7654632, 5.425437199999999 ],
                  [ 9.8588664, 2.7654632, 3.7867971999999996, 11.9889764, 7.3350832, 5.9163172 ],
                  [ 10.275493800000001, -2.7531998, 4.1909372, 11.5962938, -1.8364698000000002, 5.511727199999999 ],
                  [ 9.8594838, -9.2364698, 3.7831271999999996, 11.995003800000001, -2.7531998, 5.9180472 ],
                  [ 9.5358938, -11.6364698, 3.7505871999999996, 12.335893800000001, -9.2364698, 5.950587199999999 ],
                  [ -3.1292748, -0.6631431999999999, 4.1904666, -2.2130848, 0.6576568, 5.4264466 ],
                  [ -3.8182448, -1.0684032, 3.8745865999999998, -3.1292748, 1.0651967999999998, 5.8081866 ],
                  [ -5.1551447999999995, -1.5891532000000002, 3.5433065999999998, -3.8182448, 1.5130367999999998, 6.101866599999999 ],
                  [ -6.991184799999999, -1.1094732, 3.7634065999999997, -5.1551447999999995, 1.0460368, 5.9184266 ],
                  [ -7.4073747999999995, -0.7070831999999999, 4.156856599999999, -6.991184799999999, 0.6609368000000001, 5.4331366 ],
                  [ -5.1216948, -2.1876132, 4.2614866, -3.8453448, -1.5891532000000002, 5.4575866 ],
                  [ 6.734420599999999, -0.6496558, 4.1954958, 7.650610599999999, 0.6711442, 5.431475799999999 ],
                  [ 7.650610599999999, -1.0583658, 3.7911358, 10.557180599999999, 1.0844442, 5.9206558 ],
                  [ 10.292640599999999, -1.2239658, 3.6150057999999996, 11.6015606, 1.2186642, 6.0576358 ],
                  [ 10.292640599999999, 1.2186642, 4.1298958, 11.6015606, 2.6927242, 5.4979157999999995 ],
                  [ 10.292640599999999, -2.7574958, 4.2238158, 11.6015606, -1.2239658, 5.4595658 ],
                  [ 11.6015606, -0.7892558000000001, 4.1558958, 12.0844206, 0.8107442, 5.5558958 ],
                  [ -5.1190144, -2.8618892000000002, 4.1830244, -3.7982144, -1.9456992, 5.4190044 ],
                  [ -5.5223144, -6.0224592, 3.7659643999999997, -3.3922044, -2.8618892000000002, 5.9081844 ],
                  [ -5.700624400000001, -7.292459200000001, 3.6025343999999997, -3.2579944000000003, -6.0224592, 6.0451644 ],
                  [ -5.0560944, -7.244599200000001, 4.8434244, -3.9035044, -6.0703192, 7.1207644000000005 ],
                  [ -5.1023244000000005, -7.4794592, 6.7986144, -3.8149044, -6.191939199999999, 7.5009444 ],
                  [ -4.8312544, -7.2083692, 7.3275144, -4.0859744000000005, -6.463029199999999, 8.0728244 ] ]

In [ ]:
def plot_modules(ax):
    """
    Plot ISS module outlines (keepin boxes) as viewed from above.
    """
    for box in KEEPINS:
        xmin, ymin, zmin, xmax, ymax, zmax = box
        w = ymax - ymin
        h = xmax - xmin
        
        # sanity check bounding box geometry
        assert w >= 0
        assert h >= 0
        assert zmax >= zmin
        assert zmin >= 0

        ax.add_patch(patches.Rectangle((ymin, xmin), w, h, edgecolor="blue", fill=False))

In [ ]:
# plot ISS module outlines by themselves
fig, ax = plt.subplots()
plot_modules(ax)
plt.xlabel("y")
plt.ylabel("x")
plt.axis("tight")
plt.axis("equal")

In [ ]:
def plot_wifi(df):
    """
    Plot Wi-Fi signal strength measurements, as viewed from above ISS. Color represents signal strength.
    """
    x = df["loc_pose__pose_position_x"]
    y = df["loc_pose__pose_position_y"]
    signal = df["hw_wifi__signals0_signal_dbm"]
    fig, ax = plt.subplots()
    plt.scatter(y, x, c=signal)
    plot_modules(ax)
    plt.xlabel("y")
    plt.ylabel("x")
    plt.colorbar(label="Signal strength (dBm)")
    plt.axis("equal")
    plt.grid()
    plt.tight_layout()
    fig.set_size_inches((10,10))

In [ ]:
# List available CSV files. Each file is extracted from an Astrobee telemetry bag file.
csvs = glob.glob("20*/*/*.csv")
csvs

In [ ]:
# Read all the CSV files and concatenate the data to form a single dataset.
dfs = [pd.read_csv(c) for c in csvs]
df = pd.concat(dfs)

In [ ]:
# List CSV column data types.
df.dtypes

In [ ]:
# Show brief data summary.
df

In [ ]:
# Limit number of points so plots finish faster. Points will be drawn randomly from complete dataset.
NUM_PTS_TO_PLOT = 10000

In [ ]:
# this plot shows there are some crazy position errors (you can see ISS module outlines at bottom left)
plot_wifi(df.sample(NUM_PTS_TO_PLOT))

In [ ]:
def in_box(df, box):
    xmin, ymin, zmin, xmax, ymax, zmax = box
    fprefix = "loc_pose__pose_position_"
    xf = fprefix + "x"
    yf = fprefix + "y"
    zf = fprefix + "z"
    return df[xf].between(xmin, xmax) & df[yf].between(ymin, ymax) & df[zf].between(zmin, zmax)

def in_any_box(df, boxes):
    result = in_box(df, boxes[0])
    for box in boxes[1:]:
        result = result | in_box(df, box)
    return result

In [ ]:
# filter data to positions inside our pre-defined ISS module boundaries (get rid of crazy position errors)
df = df[in_any_box(df, KEEPINS)]

In [ ]:
plot_wifi(df.sample(NUM_PTS_TO_PLOT))